# Feature Engineering

En este notebook se buscan atributos nuevos para concatenar al set de datos original, así pudiendo armar un modelo predictivo más robusto y eficiente.



In [1]:
# Importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default')

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_train_labels = pd.read_csv('../data/train_labels.csv', low_memory=False)
df_train_values = pd.read_csv('../data/train_values.csv', low_memory=False)

In [3]:
df_buildings = pd.merge(df_train_labels, df_train_values, how='inner', on='building_id')
df_buildings.set_index('building_id', inplace=True)
df_buildings.head()

,damage_grade,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,3,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
28830,2,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
94947,3,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
590882,2,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
201944,3,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# Conversion de tipos de datos
to_categorical = set(df_buildings.select_dtypes(include='object').columns.tolist())
to_bool = set([col for col in df_buildings.columns if 'has' in col])
to_numeric = set(df_buildings.select_dtypes(include='int64').columns.tolist()) - to_bool

for col in to_categorical:
    df_buildings[col] = df_buildings[col].astype('category')

for col in to_bool:
    df_buildings[col] = df_buildings[col].astype('bool')

for col in to_numeric:
    df_buildings[col] = pd.to_numeric(df_buildings[col], downcast='unsigned')


In [5]:
df_buildings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 39 columns):
 #   Column                                  Non-Null Count   Dtype   
---  ------                                  --------------   -----   
 0   damage_grade                            260601 non-null  uint8   
 1   geo_level_1_id                          260601 non-null  uint8   
 2   geo_level_2_id                          260601 non-null  uint16  
 3   geo_level_3_id                          260601 non-null  uint16  
 4   count_floors_pre_eq                     260601 non-null  uint8   
 5   age                                     260601 non-null  uint16  
 6   area_percentage                         260601 non-null  uint8   
 7   height_percentage                       260601 non-null  uint8   
 8   land_surface_condition                  260601 non-null  category
 9   foundation_type                         260601 non-null  category
 10  roof_type                  

In [20]:
df_buildings.isnull().sum()

damage_grade                              0
geo_level_1_id                            0
geo_level_2_id                            0
geo_level_3_id                            0
count_floors_pre_eq                       0
age                                       0
area_percentage                           0
height_percentage                         0
land_surface_condition                    0
foundation_type                           0
roof_type                                 0
ground_floor_type                         0
other_floor_type                          0
position                                  0
plan_configuration                        0
has_superstructure_adobe_mud              0
has_superstructure_mud_mortar_stone       0
has_superstructure_stone_flag             0
has_superstructure_cement_mortar_stone    0
has_superstructure_mud_mortar_brick       0
has_superstructure_cement_mortar_brick    0
has_superstructure_timber                 0
has_superstructure_bamboo       

## Dropeamos columna de has_secondary_use

La columna has_secondary_use ya vimos que tiene informacion redundante con las de has_secondary_use_X

In [6]:
def drop_features_redundantes(df, features):
    df.drop(columns=features, inplace=True)

In [7]:
features_redundantes = ['has_secondary_use']
drop_features_redundantes(df_buildings, features_redundantes)

## Creacion de nuevas features a partir de columnas actuales

### Trabajando con los geo_level_ids

Sabemos que los geo_level_ids tienen una alta cardinalidad para los tres tipos de IDs. Buscamos aplicar probabilidad condicional de manera de obtener, por ejemplo, 
$ P(damage\_grade = 1 \ | \ geo\_level\_1 = 14) $. Por ende, por cada columna de geo_level_id obtendriamos 3 nuevas features creando en total 9 nuevas columnas

In [8]:
# Vemos que cada columna suma 1.
test = pd.crosstab(df_buildings['damage_grade'], df_buildings['geo_level_1_id'], normalize='columns')
test

geo_level_1_id,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
damage_grade,,,,,,,,,,,,,,,,,,,,,
1,0.08,0.15,0.09,0.03,0.04,0.17,0.09,0.05,0.03,0.14,...,0.02,0.13,0.06,0.21,0.08,0.35,0.04,0.00,0.02,0.09
2,0.77,0.73,0.66,0.60,0.77,0.75,0.67,0.59,0.45,0.69,...,0.39,0.74,0.69,0.69,0.78,0.56,0.48,0.59,0.88,0.79
3,0.15,0.11,0.25,0.36,0.20,0.09,0.25,0.35,0.52,0.17,...,0.58,0.13,0.25,0.10,0.14,0.09,0.48,0.41,0.10,0.11


In [9]:
test[0]

damage_grade
1                   0.08
2                   0.77
3                   0.15
Name: 0, dtype: float64

In [10]:
# TODO: Antes de esto podriamos chequear la cardinalidad del geo level 3 y geo level 2 pq tienen muchos valores posibles
# los IDs que tienen pocos valores los podriamos agrupar
def calcular_proba_condicional_segun_geo_level_id(df, feature_geo_level):
    proba_segun_danio_1 = []
    proba_segun_danio_2 = []
    proba_segun_danio_3 = []
    probas_condicionales = pd.crosstab(df['damage_grade'], df[feature_geo_level], normalize='columns')
    for _id in df[feature_geo_level]:
        proba_segun_danio_1.append(probas_condicionales[_id][1])
        proba_segun_danio_2.append(probas_condicionales[_id][2])
        proba_segun_danio_3.append(probas_condicionales[_id][3])
    label_spliteado = feature_geo_level.split('_')
    feature_geo_level = label_spliteado[0] + label_spliteado[1] + label_spliteado[2]
    df['proba_danio_1_dado_' + feature_geo_level] = proba_segun_danio_1
    df['proba_danio_2_dado_' + feature_geo_level] = proba_segun_danio_2
    df['proba_danio_3_dado_' + feature_geo_level] = proba_segun_danio_3        

In [11]:
# Tarda un poquito
calcular_proba_condicional_segun_geo_level_id(df_buildings, 'geo_level_1_id')
calcular_proba_condicional_segun_geo_level_id(df_buildings, 'geo_level_2_id')
calcular_proba_condicional_segun_geo_level_id(df_buildings, 'geo_level_3_id')

In [12]:
df_buildings.head()

,damage_grade,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_other,proba_danio_1_dado_geolevel1,proba_danio_2_dado_geolevel1,proba_danio_3_dado_geolevel1,proba_danio_1_dado_geolevel2,proba_danio_2_dado_geolevel2,proba_danio_3_dado_geolevel2,proba_danio_1_dado_geolevel3,proba_danio_2_dado_geolevel3,proba_danio_3_dado_geolevel3
building_id,,,,,,,,,,,,,,,,,,,,,
802906,3,6,487,12198,2,30,6,5,t,r,...,False,0.09,0.67,0.25,0.00,0.25,0.74,0.00,0.16,0.84
28830,2,8,900,2812,2,10,8,7,o,r,...,False,0.03,0.45,0.52,0.01,0.49,0.50,0.06,0.81,0.12
94947,3,21,363,8973,2,10,5,5,t,r,...,False,0.02,0.39,0.58,0.08,0.32,0.60,0.03,0.36,0.61
590882,2,22,418,10694,2,10,6,5,t,r,...,False,0.13,0.74,0.13,0.02,0.85,0.13,0.03,0.84,0.13
201944,3,11,131,1488,3,30,8,9,t,r,...,False,0.05,0.57,0.38,0.03,0.59,0.38,0.01,0.61,0.38


### Año de edificacion

In [13]:
def calcular_anio_edificacion(df):
    # Los datos fueron recolectados en 2015
    df['anio_edificacion'] = 2015 - df['age']

In [14]:
calcular_anio_edificacion(df_buildings)
df_buildings.head()

,damage_grade,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,proba_danio_1_dado_geolevel1,proba_danio_2_dado_geolevel1,proba_danio_3_dado_geolevel1,proba_danio_1_dado_geolevel2,proba_danio_2_dado_geolevel2,proba_danio_3_dado_geolevel2,proba_danio_1_dado_geolevel3,proba_danio_2_dado_geolevel3,proba_danio_3_dado_geolevel3,anio_edificacion
building_id,,,,,,,,,,,,,,,,,,,,,
802906,3,6,487,12198,2,30,6,5,t,r,...,0.09,0.67,0.25,0.00,0.25,0.74,0.00,0.16,0.84,1985
28830,2,8,900,2812,2,10,8,7,o,r,...,0.03,0.45,0.52,0.01,0.49,0.50,0.06,0.81,0.12,2005
94947,3,21,363,8973,2,10,5,5,t,r,...,0.02,0.39,0.58,0.08,0.32,0.60,0.03,0.36,0.61,2005
590882,2,22,418,10694,2,10,6,5,t,r,...,0.13,0.74,0.13,0.02,0.85,0.13,0.03,0.84,0.13,2005
201944,3,11,131,1488,3,30,8,9,t,r,...,0.05,0.57,0.38,0.03,0.59,0.38,0.01,0.61,0.38,1985


### Features basados en cantidad de pisos, area, altura y cantidad de familias

In [17]:
def features_basadas_en_cantidad_de_pisos(df):
    df['cant_pisos_por_antiguedad'] = df['count_floors_pre_eq']/ df['age']
    df['cant_pisos_por_area'] = df['count_floors_pre_eq'] / df['area_percentage']
    df['cant_pisos_por_altura'] = df['count_floors_pre_eq'] / df['height_percentage']
    
def features_basadas_en_area_altura(df):
    df['area_por_antiguedad'] = df['area_percentage'] / df['age']
    df['altura_por_antiguedad'] = df['height_percentage'] / df['age']
    df['area_por_altura'] = df['area_percentage'] / df['height_percentage']

def features_basadas_en_cantidad_familias(df):
    df['cant_familias_por_pisos'] = df['count_families'] / df['count_floors_pre_eq']
    df['cant_familias_por_area'] = df['count_families'] / df['area_percentage']
    df['cant_familias_por_altura'] = df['count_families'] / df['height_percentage']

In [19]:
features_basadas_en_cantidad_de_pisos(df_buildings)
features_basadas_en_area_altura(df_buildings)
features_basadas_en_cantidad_familias(df_buildings)
df_buildings.head()

,damage_grade,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,anio_edificacion,cant_pisos_por_antiguedad,cant_pisos_por_area,cant_pisos_por_altura,area_por_antiguedad,altura_por_antiguedad,area_por_altura,cant_familias_por_pisos,cant_familias_por_area,cant_familias_por_altura
building_id,,,,,,,,,,,,,,,,,,,,,
802906,3,6,487,12198,2,30,6,5,t,r,...,1985,0.07,0.33,0.40,0.20,0.17,1.20,0.50,0.17,0.20
28830,2,8,900,2812,2,10,8,7,o,r,...,2005,0.20,0.25,0.29,0.80,0.70,1.14,0.50,0.12,0.14
94947,3,21,363,8973,2,10,5,5,t,r,...,2005,0.20,0.40,0.40,0.50,0.50,1.00,0.50,0.20,0.20
590882,2,22,418,10694,2,10,6,5,t,r,...,2005,0.20,0.33,0.40,0.60,0.50,1.20,0.50,0.17,0.20
201944,3,11,131,1488,3,30,8,9,t,r,...,1985,0.10,0.38,0.33,0.27,0.30,0.89,0.33,0.12,0.11
